In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
ends = pd.read_csv("/Users/brentkong/Documents/curling/data/Ends.csv")
games = pd.read_csv("/Users/brentkong/Documents/curling/data/Games.csv")
stones = pd.read_csv("/Users/brentkong/Documents/curling/data/Stones.csv")

In [3]:
games["GameUID"] = games.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
stones["GameUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
stones["EndUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)
ends.loc[ends["Result"] == 9, "Result"] = 0
ends["GameUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
ends["TeamUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}", axis=1)
ends["EndUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)
ends["PowerPlay"] = ends["PowerPlay"].fillna(0)
ends["PowerPlayBool"] = ends["PowerPlay"].replace(2, 1)
ends = ends.sort_values(["GameUID", "TeamUID", "EndID"])
ends["CumulativeScore"] = ends.groupby("TeamUID")["Result"].cumsum()
stones = stones.merge(ends[["EndUID", "Result", "PowerPlay"]], on = "EndUID", how = "left").reset_index(drop = True)
ends = ends.reset_index(drop = True)

In [4]:
def assign_hammer(row):
    if row.ShotID % 2 != 0:
        return 0
    else:
        return 1

stones["Has_Hammer"] = stones.apply(assign_hammer, axis = 1)
ends = ends.merge(stones[["EndUID", "Has_Hammer"]], on="EndUID", how="left").drop_duplicates().reset_index(drop = True)

In [5]:
ends = ends.merge(games[["GameUID","NOC1","NOC2","TeamID1","TeamID2","LSFE"]], on="GameUID", how="left")

def get_opp_id(row):
    if row.TeamID == row.TeamID1:
        return row.TeamID2
    if row.TeamID == row.TeamID2:
        return row.TeamID1
    return None

ends["OpponentID"] = ends.apply(get_opp_id, axis=1)

In [6]:
ends["OpponentEndUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.OpponentID)}_{int(r.EndID)}", axis=1)

ends = ends.merge(
    ends[["EndUID", "CumulativeScore"]].rename(columns={
        "EndUID": "OpponentEndUID",
        "CumulativeScore": "OpponentCumulative"
    }),
    on="OpponentEndUID",
    how="left"
)

ends["ScoreDiff"] = ends["CumulativeScore"] - ends["OpponentCumulative"]


ends = ends.sort_values(["GameUID", "TeamID", "EndID"])

ends["PrevScoreDiff"] = (
    ends.groupby(["GameUID", "TeamID"])["ScoreDiff"]
        .shift(1)
        .fillna(0)
)


In [7]:
ends[ends["EndID"] == 1]["PrevScoreDiff"].value_counts()

PrevScoreDiff
0.0    688
Name: count, dtype: int64

In [8]:
ends = ends.merge(
    ends[["EndUID", "Result"]].rename(columns={
        "EndUID": "OpponentEndUID",
        "Result": "OpponentResult"
    }),
    on="OpponentEndUID",
    how="left"
)
ends["EndDiff"] = ends["Result"] - ends["OpponentResult"]

ends = ends.sort_values(["GameUID", "TeamID", "EndID"])

ends["PrevEndDiff"] = (
    ends.groupby(["GameUID", "TeamID"])["EndDiff"]
        .shift(1)
        .fillna(0)
)



In [9]:
ends[ends["EndID"] == 1]["PrevEndDiff"].value_counts()

PrevEndDiff
0.0    688
Name: count, dtype: int64

In [10]:
ends.columns

Index(['CompetitionID', 'SessionID', 'GameID', 'TeamID', 'EndID', 'Result',
       'PowerPlay', 'GameUID', 'TeamUID', 'EndUID', 'PowerPlayBool',
       'CumulativeScore', 'Has_Hammer', 'NOC1', 'NOC2', 'TeamID1', 'TeamID2',
       'LSFE', 'OpponentID', 'OpponentEndUID', 'OpponentCumulative',
       'ScoreDiff', 'PrevScoreDiff', 'OpponentResult', 'EndDiff',
       'PrevEndDiff'],
      dtype='object')

In [17]:
out = {}
for end in range(1, 9):
    value_counts = ends.loc[ends["EndID"] == end]["PrevScoreDiff"].value_counts().to_dict()
    for key in value_counts:
        value_counts[key] /= len(ends.loc[ends["EndID"] == end])
    out[end] = value_counts
print(out)

{1: {0.0: 1.0}, 2: {1.0: 0.27906976744186046, -1.0: 0.27906976744186046, 2.0: 0.1308139534883721, -2.0: 0.1308139534883721, 3.0: 0.055232558139534885, -3.0: 0.055232558139534885, -4.0: 0.027616279069767442, 4.0: 0.027616279069767442, 5.0: 0.005813953488372093, -5.0: 0.005813953488372093, 0.0: 0.0029069767441860465}, 3: {0.0: 0.22965116279069767, -1.0: 0.14098837209302326, 1.0: 0.14098837209302326, -2.0: 0.11627906976744186, 2.0: 0.11627906976744186, -3.0: 0.0625, 3.0: 0.0625, 4.0: 0.040697674418604654, -4.0: 0.040697674418604654, -5.0: 0.020348837209302327, 5.0: 0.020348837209302327, 6.0: 0.00436046511627907, -6.0: 0.00436046511627907}, 4: {1.0: 0.15406976744186046, -1.0: 0.15406976744186046, -2.0: 0.12645348837209303, 2.0: 0.12645348837209303, 0.0: 0.09593023255813954, 3.0: 0.09156976744186046, -3.0: 0.09156976744186046, -4.0: 0.030523255813953487, 4.0: 0.030523255813953487, 5.0: 0.023255813953488372, -5.0: 0.023255813953488372, -6.0: 0.01744186046511628, 6.0: 0.01744186046511628, 7.0

In [15]:
ends["PowerPlayBool"].unique()

array([0., 1.])

In [16]:
pp_varient = {"PP": ends.loc[ends["PowerPlayBool"] == 1],  "NO_PP": ends.loc[ends["PowerPlayBool"] == 0]}
final = {}
for action, variant in pp_varient.items():
    out = {}
    for end in range(1, 9):
        value_counts = variant.loc[variant["EndID"] == end]["EndDiff"].value_counts().to_dict()
        for key in value_counts:
            value_counts[key] /= len(variant.loc[variant["EndID"] == end])
        out[end] = value_counts
    final[action] = out
print(final)

{'PP': {1: {}, 2: {-1: 1.0}, 3: {1: 0.4666666666666667, -1: 0.2, 3: 0.13333333333333333, -2: 0.13333333333333333, 2: 0.06666666666666667}, 4: {1: 0.48148148148148145, 2: 0.18518518518518517, -1: 0.1111111111111111, -2: 0.1111111111111111, -3: 0.07407407407407407, 3: 0.037037037037037035}, 5: {1: 0.3246753246753247, 3: 0.22077922077922077, 2: 0.22077922077922077, -1: 0.15584415584415584, -2: 0.03896103896103896, 4: 0.025974025974025976, -3: 0.012987012987012988}, 6: {1: 0.33170731707317075, 2: 0.25853658536585367, 3: 0.16585365853658537, -1: 0.15609756097560976, -2: 0.02926829268292683, 5: 0.024390243902439025, 4: 0.014634146341463415, -3: 0.00975609756097561, 0: 0.004878048780487805, -4: 0.004878048780487805}, 7: {2: 0.3269230769230769, 1: 0.28365384615384615, 3: 0.14423076923076922, -1: 0.10576923076923077, -2: 0.052884615384615384, 4: 0.04807692307692308, -3: 0.014423076923076924, 5: 0.009615384615384616, 6: 0.009615384615384616, -4: 0.004807692307692308}, 8: {1: 0.35384615384615387,

In [ ]:
out = {}
for end in range(1, 9):
    value_counts = ends.loc[ends["EndID"] == end]["EndDiff"].value_counts().to_dict()
    for key in value_counts:
        value_counts[key] /= len(ends.loc[ends["EndID"] == end])
    out[end] = value_counts
print(out)

In [101]:
ends_df = ends[
    ['EndUID', "Has_Hammer",
            "PowerPlayBool",
            "EndID",
            "PrevScoreDiff"]
    ]


'''
games_df = games[[
    "GameUID", "NOC1",	"NOC2",	"ResultStr1", "ResultStr2",	"LSFE",	"Winner",
    "TeamID1", "TeamID2",
]]


stones_df = stones[[
    'GameUID', 'TeamID', 'EndID', 'ShotID', 'PlayerID', 'Task', 'Handle', 'Points', 'stone_1_x',
    'stone_1_y', 'stone_2_x', 'stone_2_y', 'stone_3_x', 'stone_3_y',
    'stone_4_x', 'stone_4_y', 'stone_5_x', 'stone_5_y', 'stone_6_x',
    'stone_6_y', 'stone_7_x', 'stone_7_y', 'stone_8_x', 'stone_8_y',
    'stone_9_x', 'stone_9_y', 'stone_10_x', 'stone_10_y', 'stone_11_x',
    'stone_11_y', 'stone_12_x', 'stone_12_y',
    'Has_Hammer', "PowerPlay", 'Result', 'BurialDepth', 'GuardAngle', 'ClusterIndex',
    'SideOpenness', 
]]
'''
ends_df.to_csv("/Users/brentkong/Documents/curling/processed_data/ends_processed_bayesian.csv", index=False)
#games_df.to_csv("/Users/brentkong/Documents/curling/processed_data/games_processed.csv", index=False)
#stones_df.to_csv("/Users/brentkong/Documents/curling/processed_data/stones_processed.csv", index=False)